In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import json
import os

## Load the 65M data

In [3]:
data = pd.read_csv("/Users/beneverman/Documents/Coding/QuantHive/IDVF-Oxford-v1/data/processed-5yr-93-minute/65min.csv", index_col=0)
# assuming 6 65-minute periods per day
back_day = 6*20 # 20 days
window_length = 6*250 # 250 days
train_size = 6*1000 # 1000 days

In [7]:
data.ffill(inplace=True)
data.bfill(inplace=True)

In [11]:
data['datetime'] = pd.to_datetime(data['datetime'])

/var/folders/6t/m1cj5jqd1cx9xq5bxs_h3g200000gn/T/ipykernel_56961/2412623082.py:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise a warning unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  data['datetime'] = pd.to_datetime(data['datetime'])


In [ ]:
expected_num_intervals = get_ny

In [12]:
data

,datetime,TMO,ABT,HD,MCD,PG,CAT,DIS,CCI,JNJ,...,COST,IBM,ADP,AVGO,WMT,AMGN,INTU,AXP,MMC,CB
0,2018-10-11 03:15:00-04:00,226.500000,68.660000,191.110000,167.120000,80.630000,142.670000,112.000000,108.565240,136.030000,...,220.720000,142.000000,140.370000,229.000000,94.733333,199.260000,201.180000,102.500000,80.916333,131.044347
1,2018-10-11 04:20:00-04:00,226.500000,68.660000,191.110000,167.120000,80.400000,142.000000,112.000000,108.565240,136.030000,...,220.720000,142.000000,140.370000,229.000000,94.733333,199.260000,201.180000,102.500000,80.916333,131.044347
2,2018-10-11 05:25:00-04:00,226.500000,68.660000,191.110000,167.120000,80.605000,142.000000,112.000000,108.565240,136.030000,...,220.720000,142.000000,140.370000,229.000000,94.733333,199.260000,201.180000,102.500000,80.916333,131.044347
3,2018-10-11 06:30:00-04:00,226.500000,68.660000,193.168889,167.120000,80.923333,141.810000,111.692000,108.565240,136.352500,...,220.720000,142.000000,140.370000,229.410000,94.733333,199.260000,201.050000,102.500000,80.916333,131.044347
4,2018-10-11 07:35:00-04:00,227.375000,67.923333,193.071523,167.050000,80.867508,142.342222,111.937778,108.565240,136.842000,...,222.166020,141.727990,140.370000,230.774950,94.898469,200.000000,202.663333,102.800000,80.916333,131.044347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40416,2023-10-09 11:15:00-04:00,492.623229,96.060248,291.353517,248.305385,142.079060,267.553906,83.865038,92.535465,157.865618,...,553.381444,141.151622,248.171073,842.619510,154.054972,270.559827,529.747426,148.938384,192.580945,209.933361
40417,2023-10-09 12:20:00-04:00,493.120755,95.974508,292.394471,248.022531,142.391108,268.041072,84.117395,92.731840,157.734743,...,554.664403,141.148651,248.396193,847.174129,154.263491,269.776813,530.851566,148.575891,192.528934,209.333689
40418,2023-10-09 13:25:00-04:00,494.221569,96.408746,294.125342,249.282913,142.878426,270.047663,84.598786,93.278042,158.218345,...,556.428335,141.768683,249.424154,853.069179,154.897403,271.512848,533.837367,149.036523,193.043211,209.947760
40419,2023-10-09 14:30:00-04:00,496.824035,96.653774,294.938383,249.341345,143.063211,270.866620,84.698043,93.284500,158.367526,...,558.444012,141.975268,250.073115,853.727239,155.391900,271.649392,534.745669,149.144355,193.399169,210.186178


In [9]:
namelist = data.columns.tolist()

,datetime,TMO,ABT,HD,MCD,PG,CAT,DIS,CCI,JNJ,...,COST,IBM,ADP,AVGO,WMT,AMGN,INTU,AXP,MMC,CB
0,2018-10-11 03:15:00-04:00,226.500000,68.660000,191.110000,167.120000,80.630000,142.670000,112.000000,108.565240,136.030000,...,220.720000,142.000000,140.370000,229.000000,94.733333,199.260000,201.180000,102.500000,80.916333,131.044347
1,2018-10-11 04:20:00-04:00,226.500000,68.660000,191.110000,167.120000,80.400000,142.000000,112.000000,108.565240,136.030000,...,220.720000,142.000000,140.370000,229.000000,94.733333,199.260000,201.180000,102.500000,80.916333,131.044347
2,2018-10-11 05:25:00-04:00,226.500000,68.660000,191.110000,167.120000,80.605000,142.000000,112.000000,108.565240,136.030000,...,220.720000,142.000000,140.370000,229.000000,94.733333,199.260000,201.180000,102.500000,80.916333,131.044347
3,2018-10-11 06:30:00-04:00,226.500000,68.660000,193.168889,167.120000,80.923333,141.810000,111.692000,108.565240,136.352500,...,220.720000,142.000000,140.370000,229.410000,94.733333,199.260000,201.050000,102.500000,80.916333,131.044347
4,2018-10-11 07:35:00-04:00,227.375000,67.923333,193.071523,167.050000,80.867508,142.342222,111.937778,108.565240,136.842000,...,222.166020,141.727990,140.370000,230.774950,94.898469,200.000000,202.663333,102.800000,80.916333,131.044347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40416,2023-10-09 11:15:00-04:00,492.623229,96.060248,291.353517,248.305385,142.079060,267.553906,83.865038,92.535465,157.865618,...,553.381444,141.151622,248.171073,842.619510,154.054972,270.559827,529.747426,148.938384,192.580945,209.933361
40417,2023-10-09 12:20:00-04:00,493.120755,95.974508,292.394471,248.022531,142.391108,268.041072,84.117395,92.731840,157.734743,...,554.664403,141.148651,248.396193,847.174129,154.263491,269.776813,530.851566,148.575891,192.528934,209.333689
40418,2023-10-09 13:25:00-04:00,494.221569,96.408746,294.125342,249.282913,142.878426,270.047663,84.598786,93.278042,158.218345,...,556.428335,141.768683,249.424154,853.069179,154.897403,271.512848,533.837367,149.036523,193.043211,209.947760
40419,2023-10-09 14:30:00-04:00,496.824035,96.653774,294.938383,249.341345,143.063211,270.866620,84.698043,93.284500,158.367526,...,558.444012,141.975268,250.073115,853.727239,155.391900,271.649392,534.745669,149.144355,193.399169,210.186178
